### 4. 重排序
- https://www.53ai.com/news/qianyanjishu/2072.html
- https://luxiangdong.com/2024/02/22/advrag4/

### 5. RAG Prompt 设计
- https://smith.langchain.com/hub
- rlm/rag-prompt

In [20]:
# RAG中A的过程，可以在此处修改
prompt = ChatPromptTemplate.from_messages([
  ("human", 
"""
你是一个执行问答任务的智能助手。
请使用下面检索出来的上下文片段来回答问题。
如果上下文中没有相关的答案，请使用你自己的知识回答问题。
保证结果的简洁性。
Question: {question} 
Context: {context} 
Answer:
"""),
])

### 5. 搭建RAG链

In [21]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [22]:
def format_docs(raw_docs):
    """
        将所有文档的内容拼接成一个字符串
    """
    # rerank 过程
    passages = [doc.page_content for doc in raw_docs]
    rerank_results = reranker.rerank(query=query, passages=passages)
    idxes = rerank_results["rerank_ids"][:3]
    rerank_docs = [raw_docs[idx] for idx in idxes]
    
    # reorder 过程
    final_docs = recorder.transform_documents(documents=rerank_docs)

    # 拼接所有 doc 成为一个段落
    context = "\n\n".join(doc.page_content for doc in final_docs)
    return context

In [23]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

### 6. 结果测试

In [24]:
# 提问相关问题
rag_chain.invoke(input="这款口服液叫什么？")

09/18/2024 15:39:44 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:8080/api/v1/collections/34d9f664-86c0-4fa3-87a3-9a541c6ec437/query "HTTP/1.1 200 OK"
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


'大聪明口服液'

In [25]:
# 提问无关问题
rag_chain.invoke(input="抗日战争是哪年结束的？")

09/18/2024 15:39:47 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:8080/api/v1/collections/34d9f664-86c0-4fa3-87a3-9a541c6ec437/query "HTTP/1.1 200 OK"


'抗日战争是1945年结束的。'